In [10]:
import os
import shutil

import matplotlib.pyplot as plt
import pandas as pd
import scipy

from utils.dlt import *
from utils.motion_visualize_tool import *
from utils.skeleton_keypoints import *

from IPython.display import display

In [3]:
output_dir = "28"
os.makedirs(output_dir, exist_ok=True)

In [ ]:
pred_fps = 60
true_fps = 250

df_lag = pd.read_csv("lag.csv")

In [20]:
for data_num in range(10):
    true_path = f"fixed_trajectories/hirasaki_{data_num}_trajectories.csv"
    df_true = pd.read_csv(true_path, index_col=0)
    df_true["time"] = np.arange(len(df_true)) / true_fps

    pred_path = f"all_combinations/hirasaki_{data_num}_01234.csv"
    df_pred = pd.read_csv(pred_path, index_col=0)
    df_pred["time"] = np.arange(len(df_pred)) / pred_fps

    lag = df_lag["lag_RIGHT_ANKLE"].values[data_num]
    lag_time = df_lag["lag_time_RIGHT_ANKLE"].values[data_num]

    pred_time = df_pred.time.values
    true_time = df_true.time.values - lag_time

    lower = np.where(pred_time >= pred_time[np.where(pred_time > true_time[0])].min())[0]
    upper = np.where(pred_time <= pred_time[np.where(pred_time < true_time[-1])].max())[0]
    common_index = np.intersect1d(lower, upper)
    new_time = pred_time[common_index]

    df_true2 = pd.DataFrame()
    for keypoint in df_true.columns:
        if keypoint == "time":
            continue
        y_true = df_true[keypoint]
        interpolater = scipy.interpolate.interp1d(true_time, y_true, kind="linear")
        y_true_interpolated = interpolater(new_time)
        df_true2[keypoint] = y_true_interpolated
    df_true2["time"] = new_time
    df_true2["frame"] = (df_true2["time"] * pred_fps + 0.5).astype(int) + 1
    df_true2.set_index("frame", inplace=True)
    
    df_true2.to_csv(f"{output_dir}/hirasaki_{data_num}_trajectories.csv")